In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
IMG_PATH="../facial_expressions/"

In [4]:
emotions=["anger", "disgust", "fear", "happy", "neutral", "sad"]

In [5]:
face_dim=128

In [6]:
import matplotlib.pyplot as plt

In [7]:
def ret_keypoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)==0:
        param=[0,0,face_dim,face_dim]
    else:
        (x,y,w,h)=faces[0]
        param=[x,y,x+w,y+h]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [8]:
def prep_keypoints():
    X_inp=[]
    y_inp=[]
    for em in emotions:
        ltt=os.listdir(IMG_PATH+em)
        lnltt=len(ltt)
        for idx,imn in enumerate(ltt):
            print("\r",idx+1,'/',lnltt,end=" ")
            image = cv2.imread(IMG_PATH+em+"/"+imn)
            landmarks=ret_keypoints(image)
            if landmarks is not None:
                X_inp.append(landmarks)
                yy=np.zeros(len(emotions))
                yy[emotions.index(em)]=1
                y_inp.append(yy)
        print()
    return np.asarray(X_inp),np.asarray(y_inp)

In [9]:
XXD,YYD=prep_keypoints()

 124 / 124 
 84 / 84 
 67 / 67  67 
 127 / 127 
 123 / 123 
 132 / 132 


In [10]:
s=np.arange(len(XXD))
for i in range(5):
    np.random.shuffle(s)
    XXD=XXD[s]
    YYD=YYD[s]

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [12]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(emotions),activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              140288    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
cut=80
XDB=XXD[cut:]
YDB=YYD[cut:]

XTB=XXD[:cut]
YTB=YYD[:cut]

In [22]:
model.fit(XDB,YDB,batch_size=32,epochs=100,validation_data=(XTB,YTB))

Train on 577 samples, validate on 80 samples
Epoch 1/100
577/577 [==============================] - 0s 406us/sample - loss: 0.6627 - accuracy: 0.7470 - val_loss: 2.6617 - val_accuracy: 0.3125
Epoch 2/100
577/577 [==============================] - 0s 404us/sample - loss: 0.6858 - accuracy: 0.7331 - val_loss: 2.2152 - val_accuracy: 0.3625
Epoch 3/100
577/577 [==============================] - 0s 430us/sample - loss: 0.6915 - accuracy: 0.7487 - val_loss: 1.4944 - val_accuracy: 0.4875
Epoch 4/100
577/577 [==============================] - 0s 401us/sample - loss: 0.6633 - accuracy: 0.7452 - val_loss: 1.0188 - val_accuracy: 0.7125
Epoch 5/100
577/577 [==============================] - 0s 429us/sample - loss: 0.6084 - accuracy: 0.7764 - val_loss: 2.1063 - val_accuracy: 0.3375
Epoch 6/100
577/577 [==============================] - 0s 456us/sample - loss: 0.5865 - accuracy: 0.7660 - val_loss: 1.9143 - val_accuracy: 0.4250
Epoch 7/100
577/577 [==============================] - 0s 439us/sample - 

577/577 [==============================] - 0s 401us/sample - loss: 0.5580 - accuracy: 0.7990 - val_loss: 2.9871 - val_accuracy: 0.2875
Epoch 57/100
577/577 [==============================] - 0s 416us/sample - loss: 0.5428 - accuracy: 0.8007 - val_loss: 3.3723 - val_accuracy: 0.3750
Epoch 58/100
577/577 [==============================] - 0s 397us/sample - loss: 0.5878 - accuracy: 0.7886 - val_loss: 5.5034 - val_accuracy: 0.2000
Epoch 59/100
577/577 [==============================] - 0s 385us/sample - loss: 0.5482 - accuracy: 0.8059 - val_loss: 1.4419 - val_accuracy: 0.5500
Epoch 60/100
577/577 [==============================] - 0s 394us/sample - loss: 0.4703 - accuracy: 0.8423 - val_loss: 2.2443 - val_accuracy: 0.4875
Epoch 61/100
577/577 [==============================] - 0s 403us/sample - loss: 0.4638 - accuracy: 0.8146 - val_loss: 1.6149 - val_accuracy: 0.4750
Epoch 62/100
577/577 [==============================] - 0s 391us/sample - loss: 0.5058 - accuracy: 0.8215 - val_loss: 1.5256 

In [23]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.625

In [24]:
model.predict(XTB).argmax(axis=1)

array([0, 2, 2, 5, 5, 0, 0, 5, 5, 0, 4, 4, 4, 3, 4, 5, 4, 0, 4, 4, 0, 0,
       3, 3, 3, 4, 0, 3, 3, 2, 0, 0, 0, 2, 4, 0, 2, 0, 5, 4, 2, 4, 3, 4,
       0, 4, 0, 0, 0, 5, 0, 2, 5, 2, 0, 0, 0, 5, 2, 2, 0, 0, 3, 2, 4, 3,
       0, 2, 3, 0, 0, 3, 4, 0, 4, 5, 4, 4, 4, 4])

In [25]:
YTB.argmax(axis=1)

array([1, 3, 2, 5, 5, 0, 0, 5, 3, 0, 4, 4, 4, 3, 4, 5, 4, 1, 4, 4, 3, 0,
       3, 3, 3, 4, 1, 3, 3, 5, 5, 3, 5, 5, 4, 1, 2, 0, 5, 4, 5, 1, 3, 4,
       0, 4, 2, 0, 0, 5, 5, 3, 2, 2, 0, 0, 2, 5, 5, 2, 3, 1, 3, 4, 4, 3,
       2, 3, 3, 5, 1, 3, 4, 5, 4, 0, 1, 4, 4, 4])

In [20]:
cam = cv2.VideoCapture(0)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    landmarks=[]
    for (x,y,w,h) in faces:
        gray=gray[y:y+h,x:x+w]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            print(gray,x,y,w,h)
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 2, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
    if len(landmarks)>0:
        y_out=model.predict(np.asarray(landmarks))
        res=np.argmax(y_out,axis=1)
        for r in res:
            cv2.putText(img,emotions[r],(xp,yp),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

[] 308 234 120 120
[] 311 235 120 120
[] 310 233 120 120
[] 308 236 116 116
[] 314 236 119 119
[] 315 238 116 116
[] 315 237 116 116
[] 328 234 123 123
[] 337 192 156 156
[] 340 193 190 190


In [21]:
cam.release()
cv2.destroyAllWindows()